# Projeto 3
## André Ejzenmesser
## Vinicius Bonelli

Variáveis escolhidas:

Variável resposta: Indice de percepção de corrupção (Corruption Perception Index - CPI)

Outras: 

In [1]:
%matplotlib inline

import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import pandas as pd
import numpy as np

import statsmodels.api as sm1 #Usa OLS com X como matriz de planejamento
import statsmodels.formula.api as sm  #Usa OLS como no R

print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/AndreEjz/Desktop/CDP3


In [2]:
d1 = pd.read_excel('Dados.xlsx')
d2 = pd.read_excel('Corrupcao.xlsx')